# Covariances at Cls level

First we compute the covariances matrices. Then we recombined the subbands the same way as we do at the map level

In [ ]:
%matplotlib inline
import seaborn as sns
import os
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, combinations_with_replacement

# Specific qubic modules
import qubic
from qubic.polyacquisition import compute_freq
from qubic import AnalysisMC as amc
from qubic import NamasterLib as nam

rc('figure', figsize=(13, 10))
rc('font', size=18)


## Get the cross spectra

We can choose: 
- cross spectra from full pipeline simu (small number of realisations ~30)
- cross spectra from Fast Simulators (lots of realisation ~1500)

In both cases, they are cross spectra on residuals so they should be 0.


In [ ]:
#Set paths
machine = os.uname()[1]
def check_dir(dirfile):
    if os.path.isdir( dirfile): 
        print('dirfile:', dirfile); 
    else: 
        raise ValueError('dirfile {} is not an existing directory'.format(dirfile) )
        
if 'moussetDell' in machine:
    datadir = os.environ['DATA_SPECTROIM']
    rep_fullpipe = datadir + '/NERSC_simuEnd2End/Bad_photon_noise2020/simu_NERSC_MC/IBCS_fullpipeline/'
    rep_spectrumCC = datadir + 'FastSimulator/cls_spectrum_fromCC/nbands6-7/'
    check_dir(dirfiles)
elif machine == 'hain':
    datadir = '/home/martin/QUBIC/qubic/qubic/'
    rep_spectrumCC = datadir + 'data/LouiseSims/IBCS_fastsim/'
    #rep_fullpipe = datadir + 'data/LouiseSims/IBCS_fastsim/'
    ##rep_spectrumCC = datadir + ''
    check_dir(rep_spectrumCC)
else:
    raise ValueError("*dirfile variable not defined*. You are in machine named: {}. \
    You have to specify a directory where the simulations of spectra are saved,\
    e.g. [NERSC directory] /sps/qubic/Users/lmousset/SpectroImaging/FastSimMC/v4".format(machine))
    
    
print(rep_spectrumCC)


In [ ]:
# Convert all files to one .npy
# Do it just once
# freq = 220
# map_type = 'qubic'
# IBCS_type = 'mix'
# for nfrecon in [6, 7]:
#     list_files = glob.glob(rep_spectrumCC + f'IBCS{IBCS_type}*nfrecon{nfrecon}*_{map_type}maps_*FI{freq}*_v4_gal*.npy')
#     nfiles = len(list_files)
#     print('# Files:', nfiles)
#     for i in range(nfiles):
#         cross = np.load(list_files[i])
# #         with open(list_files[i], 'rb') as f:
# #             cross = pickle.load(f)

#         if i == 0:
#             allcross = cross
#         else:
#             allcross = np.concatenate((allcross, cross), axis=0)
#     print('All cross shape:', allcross.shape)
#     np.save(rep_spectrumCC + 
#             f'IBCS{IBCS_type}_nfrecon{nfrecon}_{map_type}maps_FI{freq}_v4_galaxycenter.npy', allcross)

In [ ]:
clnames = ['TT', 'EE', 'BB', 'TE']
nfrecon = 4

# Full pipeline
# cls_cross = np.load(rep_fullpipe + 'IBCSsame_recon_{}bands_150fullpipeline.npy'.format(nfrecon))

# Fast Simulator
# cls_cross = np.load(rep_spectrumCC + 
#                     'cross_interband_samereal_nfrecon{}_noisemaps_FI150_v2_galaxycenter.npy'.format(nfrecon))
cls_cross = np.load(rep_spectrumCC + 
                    'IBCSsame_nfrecon{}_qubicmaps_FI150_v4_galaxycenter.npy'.format(nfrecon))


print(cls_cross.shape)

nreals = cls_cross.shape[0]
nbins = cls_cross.shape[2]

print('# realisations:', nreals)
print('# bins:', nbins)

In [ ]:
combi = list(combinations_with_replacement(np.arange(nfrecon), 2))
print(combi)
ncombi = len(combi)

combi1 = [(i, i) for i in range(nfrecon)]
combi2 = list(combinations(np.arange(nfrecon), 2))
oldcombi = combi1 + combi2
print(oldcombi)

# Reorder cross spectra, from 00 01 02 11 12 22 to 00 11 22 01 02 12
# Only if you want to see the block with auto spectra in the covariance Matrix
permutation = []
for i in oldcombi:
    permutation.append(combi.index(i))
print(permutation)

print(cls_cross[0, :, :2, 0])
cls_cross = cls_cross[:, permutation, :, :]
print(cls_cross[0, :, :2, 0])

# Color palet for plots
palet = sns.cubehelix_palette(ncombi, start=3, hue=1, light=0.75)
sns.palplot(palet)

In [ ]:
# Plot one spectrum and the mean (just to see)
# Create a Namaster object
nside = 256
lmin = 40
lmax = 2 * nside - 1
delta_ell = 30
print('lmin', lmin)
print('lmax', lmax)
print('delta ell', delta_ell)

mask = np.zeros(12 * nside**2)

Namaster = nam.Namaster(mask, lmin=lmin, lmax=lmax, delta_ell=delta_ell)

# Binning
ell_binned, b = Namaster.get_binning(nside)
nbins = len(ell_binned)
print('nbins:', nbins)
print('ell binned:', ell_binned)

# Average them
cross_mean_residuals = np.mean(cls_cross, axis=0)
cross_std_residuals = np.std(cls_cross, axis=0) * np.sqrt(2)


clnames = ['TT', 'EE', 'BB', 'TE']
rc('figure', figsize=(12, 8))

fig, axs = plt.subplots(2, 2)
axs = axs.ravel()

for i in range(4):
    ax = axs[i]
    for cross in range(ncombi):
        c = palet[cross]
        ax.errorbar(ell_binned[:], 
                cross_mean_residuals[cross, :, i],
                yerr=cross_std_residuals[cross, :, i],
                marker='o', color=c, linestyle='none', 
                label='band {}'.format(combi[cross]))
#         ax.errorbar(ell_binned[:], 
#                 cls_cross[0, cross, :, i] - cross_mean_residuals[cross, :, i],
#                 yerr=cross_std_residuals[cross, :, i] * np.sqrt(2),
#                 marker='o', color=colorlist[cross], linestyle='none', 
#                 label='band {}'.format(combi[cross]))

    ax.set_xlabel('$\\ell$')
    ax.set_ylabel('$D_\\ell$')
    ax.set_title(clnames[i])
    #ax.set_xscale('log')
    #ax.set_yscale('log')
    ax.grid()
    ax.legend(fontsize=8, loc='upper left')

plt.tight_layout()



In [ ]:
nbands = 6
Nfreq_edges, nus_edge, nus, deltas, Delta, Nbbands = compute_freq(150, nbands)
print(nus)

combi_nus1 = [(i, i) for i in nus]
combi_nus2 = list(combinations(nus, 2))
combi_nus = combi_nus1 + combi_nus2
print(combi_nus)

palet1 = sns.cubehelix_palette(ncombi, start=0, hue=1, light=0.75, dark=0.3)
sns.palplot(palet1)
palet2 = sns.cubehelix_palette(ncombi, start=2, hue=1, light=0.75, dark=0.3)
sns.palplot(palet2)


In [ ]:
ncombi

In [ ]:
cls_cross_same = np.load(rep_spectrumCC + 
                    'IBCSsame_nfrecon{}_noisemaps_FI150_v4_galaxycenter.npy'.format(nfrecon))

cls_cross_mix = np.load(rep_spectrumCC + 
                    'IBCSmix_nfrecon{}_noisemaps_FI150_v4_galaxycenter.npy'.format(nfrecon))

# Reorder cross spectra
permutation = []
for i in oldcombi:
    permutation.append(combi.index(i))
    
cls_cross_same = cls_cross_same[:, permutation, :, :]
cls_cross_mix = cls_cross_mix[:, permutation, :, :]


# Average them
mean_same = np.mean(cls_cross_same, axis=0)
mean_mix = np.mean(cls_cross_mix, axis=0)

std_same = np.std(cls_cross_same, axis=0)
std_mix = np.std(cls_cross_mix, axis=0) * np.sqrt(2)


# Plot for paper: only BB
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 6))
axs = axs.ravel()

for cross in range(ncombi):
    if cross<int(ncombi/2+1):
        c = palet1[cross]
    else:
        c = palet2[cross-int(ncombi/2+1)]
    ax0.errorbar(ell_binned[:], 
            mean_same[cross, :, 2],
            yerr=std_same[cross, :, 2],
            marker='o', 
            color=c, 
            linestyle='none', 
            label='{:3.0f} x {:3.0f}'.format(combi_nus[cross][0], combi_nus[cross][1]))

ax0.set_xlabel('$\\ell$')
ax0.set_ylabel('$D_\\ell$')
ax0.set_title(clnames[2])
ax0.grid()
ax0.legend(fontsize=12, loc='upper left')

for cross in range(ncombi):
    if cross<int(ncombi/2+1):
        c = palet1[cross]
    else:
        c = palet2[cross-int(ncombi/2+1)]
    ax1.errorbar(ell_binned[:], 
            mean_mix[cross, :, 2],
            yerr=std_mix[cross, :, 2],
            marker='o', 
            color=c, 
            linestyle='none', 
            label='{:3.0f} x {:3.0f}'.format(combi_nus[cross][0], combi_nus[cross][1]))

ax1.set_xlabel('$\\ell$')
ax1.set_ylabel('$D_\\ell$')
ax1.set_title(clnames[2])
ax1.grid()
ax1.legend(fontsize=12, loc='upper left')


plt.tight_layout(pad=3)
# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/IBCS_noiseBB.pdf')

In [ ]:
### Figure spectroim paper
fig = plt.figure(figsize=(12, 8))

for cross in range(ncombi):
    if cross < nbands:
        c = palet1[cross]
    else:
        c = palet2[cross-4]
    plt.errorbar(ell_binned[:], 
            mean_same[cross, :, 2],
            yerr=std_same[cross, :, 2],
                 capsize  =3,
            marker='o', 
            color=c, 
            linestyle='none', 
            label='{:3.0f} x {:3.0f}'.format(combi_nus[cross][0], combi_nus[cross][1]))

plt.xlabel('Momento multipolar, $\\ell$')
plt.ylabel('$D_\\ell \,[\mu \mathrm{K}^2]$')
plt.title(clnames[2])
plt.grid()
plt.legend(fontsize=16, loc='upper left')
#plt.savefig('/home/martin/Doctorado/Tesis-Doctoral/Figuras_Tesis_Overleaf/IBCS_noiseBBsame150_v4')

In [ ]:
b = 7
s = 0
plt.scatter(cls_cross[:, 1, b, s], cls_cross[:, 2, b, s])
pval = np.polyfit(cls_cross[:, 1, b, s], cls_cross[:, 2, b, s], deg=1)
print(pval)
x = np.linspace(np.min(cls_cross[:, 1, b, s]), np.max(cls_cross[:, 1, b, s]), 10)
plt.plot(x, np.polyval(pval, x))
plt.title(f'{clnames[s]} - bin {b}')
plt.xlabel('Freq 0')
plt.ylabel('Freq 1')

In [ ]:
# Compare noise between 150 and 220
nfrecon = 6
combi = list(combinations_with_replacement(np.arange(nfrecon), 2))
ncombi = len(combi)

cls_cross150 = np.load(rep_spectrumCC + 
                    'IBCSmix_nfrecon{}_noisemaps_FI150_v4_galaxycenter.npy'.format(nfrecon))
cls_cross220 = np.load(rep_spectrumCC + 
                    'IBCSmix_nfrecon{}_noisemaps_FI220_v4_galaxycenter.npy'.format(nfrecon))
print(cls_cross150.shape)

# STD for each
std150 = np.std(cls_cross150, axis=0) #* np.sqrt(2)
std220 = np.std(cls_cross220, axis=0) #* np.sqrt(2)

print(std220.shape)

std150_BB = std150[:, :, 2]
std220_BB = std220[:, :, 2]

for i in range(ncombi):
    plt.step(ell_binned, std150_BB[i, :] / std220_BB[i, :], where='mid', label=f'IBCS {i}')
    plt.plot(ell_binned, std150_BB[i, :] / std220_BB[i, :], 'o')
plt.legend()
plt.grid()
plt.xlabel('$l$')
plt.ylabel('$\Delta D_l(150) / \Delta D_l(220)$')

## Get Cp

For each bin, each spectrum (TT, EE, BB, TE) we compute the covariance and correlation matrices between bands f0, f1, f2...

In [ ]:
Cp = amc.get_Cp(cls_cross, verbose=False)
print('Cp shape:', Cp.shape)

Cpcorr = np.zeros_like(Cp)
for b in range(nbins):
    for s in range(4):
        Cpcorr[:, :, s, b] = amc.cov2corr(Cp[:, :, s, b])

In [ ]:
labs = []
for i in range(ncombi):
    labs.append(f'{oldcombi[i][0]}{oldcombi[i][1]}')
print(labs)

plt.figure(figsize=(20, 100))
for b in range(nbins):
    for s in range(4):
        plt.subplot(nbins, 4, 4*b + s + 1)
        vmax = np.max(np.abs(Cp[:, :, s, b]))
        plt.imshow(Cp[:, :, s, b], vmin=-vmax, vmax=vmax, cmap='bwr')
        plt.title('Cov - ' + clnames[s] + ' - bin {}'.format(b))
        plt.colorbar(orientation='horizontal', fraction=0.2)
        plt.xticks(np.arange(len(labs)), labs)
        plt.yticks(np.arange(len(labs)), labs)

        

In [ ]:
plt.figure(figsize=(20, 100))
for b in range(nbins):
    for s in range(4):
        plt.subplot(nbins, 4, 4*b + s + 1)
        plt.imshow(Cpcorr[:, :, s, b], vmin=-1, vmax=1, cmap='bwr')
        
        plt.axhline(y=nfrecon-0.5, lw=3, color='k')
        plt.axvline(x=nfrecon-0.5, lw=3, color='k')

        plt.title('Corr - ' + clnames[s] + ' - bin {}'.format(b))
        plt.colorbar(orientation='horizontal', fraction=0.2)
        plt.xticks(np.arange(len(labs)), labs)
        plt.yticks(np.arange(len(labs)), labs)
        
  

In [ ]:
### Figure spectroim paper
Cpcorr_avg = np.mean(Cpcorr, axis=3)
plt.figure(figsize=(16, 5))

for s in range(4):
    plt.subplot(1, 4, s + 1)
    plt.imshow(Cpcorr_avg[:, :, s], vmin=-1, vmax=1, cmap='bwr')
    
    plt.axhline(y=nfrecon-0.5, lw=3, color='k')
    plt.axvline(x=nfrecon-0.5, lw=3, color='k')

    plt.title(clnames[s])
    plt.colorbar(orientation='horizontal', fraction=0.2)
    plt.xticks(np.arange(len(labs)), labs)
    plt.yticks(np.arange(len(labs)), labs)
# plt.suptitle('Correlation Avg')
# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/IBCS_CorrMatrices_FI150_AvgOverBins_GoodPhotonNoise_v4.pdf')

### Go to Cp_prime

Not sure it is useful because if we have many realisations (easy with Fast Simulator), we can use directly Cp to average the subbands. 

In [ ]:
# Cp_prime over bins (normalisation by the first term)
N00, Cp_prime00 = amc.Cp2Cp_prime(Cp, verbose=True)

# bin:
b = 10

plt.figure(figsize=(15, 10))
for s in range(4):
    plt.subplot(1, 4, s+1)
    vmax = np.max(np.abs(Cp_prime00[:, :, s, b]))
    plt.imshow(Cp_prime00[:, :, s, b], vmin=-vmax, vmax=vmax, cmap='bwr')
    plt.title(clnames[s] + ' - bin {}'.format(b))
    plt.colorbar(orientation='horizontal', fraction=0.1)
    plt.xticks(np.arange(len(labs)), labs)
    plt.yticks(np.arange(len(labs)), labs)

 

In [ ]:
# Cp_prime over bins (normalisation by the diagonal)
N, Cp_prime = amc.Cp2Cp_prime_viaCorr(Cp, verbose=True)

b = 10

plt.figure(figsize=(15, 10))
for s in range(4):
    plt.subplot(1, 4, s+1)
    vmax = np.max(np.abs(Cp_prime[:, :, s, b]))
    plt.imshow(Cp_prime[:, :, s, b], vmin=-vmax, vmax=vmax, cmap='bwr')
    plt.title(clnames[s] + f' - bin {b}')
    plt.colorbar(orientation='horizontal', fraction=0.1)
    plt.xticks(np.arange(len(labs)), labs)
    plt.yticks(np.arange(len(labs)), labs)
 

### Big covariance matrix mixing TT, EE, BB, TE 

Instead of looking at correlations in TT, EE, BB, TE separetely, we also compute correlations between the 4 spectra. 

But we see that they are very small.

In [ ]:
bigcov, bigcorr = amc.get_covcorr_patch(cls_cross, stokesjoint=True, doplot=False)
print(bigcov.shape)

In [ ]:
labs = []
for s in range(4):
    for i in range(ncombi):
        labs.append(clnames[s] + f'{combi[i][0]}{combi[i][1]}')
        
print(labs)

In [ ]:
plt.figure(figsize=(20, 200))
for b in range(nbins):
    
    plt.subplot(nbins, 2, b*2 +1)
    vmax = np.max(np.abs(bigcov[:, :, b]))
    plt.imshow(bigcov[:, :, b], vmin=-vmax, vmax=vmax, cmap='bwr')
    plt.xticks(np.arange(len(labs)), labs, fontsize=7)
    plt.yticks(np.arange(len(labs)), labs, fontsize=7)
    for s in range(nfrecon):
        plt.axhline(y=(s+1)*ncombi-0.5, lw=3, color='k')
        plt.axvline(x=(s+1)*ncombi-0.5, lw=3, color='k')
    plt.title('Covariance - bin {}'.format(b))
    plt.colorbar(orientation='horizontal')
    
    plt.subplot(nbins, 2, b*2 +2)
    plt.imshow(bigcorr[:, :, b], vmin=-1, vmax=1, cmap='bwr')
    plt.xticks(np.arange(len(labs)), labs, fontsize=9)
    plt.yticks(np.arange(len(labs)), labs, fontsize=9)
    for s in range(nfrecon):
        plt.axhline(y=(s+1)*ncombi-0.5, lw=3, color='k')
        plt.axvline(x=(s+1)*ncombi-0.5, lw=3, color='k')
    plt.title('Correlation - bin {}'.format(b))
    plt.colorbar(orientation='horizontal')

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(np.mean(bigcorr, axis=2), vmin=-1, vmax=1, cmap='bwr')
plt.title('Correlation matrice averaged over bins')
plt.colorbar(orientation='horizontal')

plt.xticks(np.arange(len(labs)), labs, fontsize=8)
plt.yticks(np.arange(len(labs)), labs, fontsize=8)

for s in range(nfrecon):
    plt.axhline(y=(s+1)*ncombi-0.5, lw=3, color='k')
    plt.axvline(x=(s+1)*ncombi-0.5, lw=3, color='k')


## Covariance matrix between bins

Now for each spectrum TT, EE, BB and TE, we compute the covariance matric between bins and subbands. 

This matrix has a shape of (nbins x nfrecon) x (nbins x nfrecon)

In [ ]:
# We have to reshape cls_cross as (nreals, nbins, 4, nfrecon)
print('Old shape:', cls_cross.shape)
cls_cross_reshape = np.moveaxis(cls_cross, [1, 2, 3], [3, 1, 2])
print('New shape:', cls_cross_reshape.shape)

In [ ]:
covbinband, corrbinband = amc.get_covcorr_patch(cls_cross_reshape, 
                                                stokesjoint=True, doplot=False)

print(nbins)
print(covbinband.shape)

In [ ]:
def clean_matrices(matrix, ncombi, nbins=16, ):
    clean_matrix = np.zeros_like(matrix)
    for i in range(-ncombi, ncombi):
        # Get the diagonal
        thediagonal = np.diag(matrix, i*nbins)
        # Make a matrix with the diagonal and 0 elsewhere
        clean_matrix += np.diag(thediagonal, i*nbins)
    return clean_matrix


In [ ]:
# Put the term out of the small diagonal to 0 as they are due to the noise.
BBcov = covbinband[:, :, 2]
BBcorr = corrbinband[:, :, 2]


# clean_BBcorr = clean_matrices(BBcorr, ncombi)
# clean_BBcov = clean_matrices(BBcov, ncombi)

In [ ]:
datadir

In [ ]:
# Get matrices and save them for all nfrecon for Claudia
rep_save = '/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/corr_matrices/pourClaudia/'
freq = 220
tipo = 'mix'

for nfrecon in [6, 7]:
    
    combi = list(combinations_with_replacement(np.arange(nfrecon), 2))
    ncombi = len(combi)

    cls_cross = np.load(rep_spectrumCC + 
                    f'IBCS{tipo}_nfrecon{nfrecon}_noisemaps_FI{freq}_v4_galaxycenter.npy')

    print(cls_cross.shape)
    
    cls_cross_reshape = np.moveaxis(cls_cross, [1, 2, 3], [3, 1, 2])
    covbinband, corrbinband = amc.get_covcorr_patch(cls_cross_reshape, stokesjoint=True, doplot=False)

       
    BBcov = covbinband[:, :, 2]
    BBcorr = corrbinband[:, :, 2]
    print(BBcov.shape)

    # Clean the matrix
#     threshold = 0.01
#     clean_BBcov = np.where(np.abs(BBcorr)<threshold, BBcov*0, BBcov)
#     clean_BBcorr = np.where(np.abs(BBcorr)<threshold, BBcorr*0, BBcorr)
     
    clean_BBcorr = clean_matrices(BBcorr, ncombi)
    clean_BBcov = clean_matrices(BBcov, ncombi)
    
    np.save(rep_save + 
            f'BBcovariance_bincross_nfrecon{nfrecon}_{tipo}real_FI{freq}_v4.npy', 
            BBcov)
    np.save(rep_save + 
            f'BBcorrelation_bincross_nfrecon{nfrecon}_{tipo}real_FI{freq}_v4.npy', 
            BBcorr)
    
#     np.save(rep_save + 
#             f'BBcovariance_bincross_nfrecon{nfrecon}_{tipo}real_FI{freq}_v3_cleanbelow{threshold}.npy', 
#             clean_BBcov)
#     np.save(rep_save + 
#             f'BBcorrelation_bincross_nfrecon{nfrecon}_{tipo}real_FI{freq}_v3_cleanbelow{threshold}.npy', 
#             clean_BBcorr)
    np.save(rep_save + 
            f'BBcovariance_bincross_nfrecon{nfrecon}_{tipo}real_FI{freq}_v4_clean.npy', 
            clean_BBcov)
    np.save(rep_save + 
            f'BBcorrelation_bincross_nfrecon{nfrecon}_{tipo}real_FI{freq}_v4_clean.npy', 
            clean_BBcorr)

In [ ]:
# Load matrice already saved
# nfrecon = 5
# combi = list(combinations_with_replacement(np.arange(nfrecon), 2))
# print(combi)
# ncombi = len(combi)
# nbins=16

### FI150
# covbinbandBB = np.load(rep_save + 'BBcovariance_bincross_nfrecon{}_samereal_FI150_v1.npy'.format(nfrecon))
# corrbinbandBB = np.load(rep_save + 'BBcorrelation_bincross_nfrecon{}_mixreal_FI150_v1.npy'.format(nfrecon))

# covbinbandBB = np.load(rep_save + 'BBcovariance_bincross_nfrecon{}_mixreal_FI220_v1.npy'.format(nfrecon))
# corrbinbandBB = np.load(rep_save + 'BBcorrelation_bincross_nfrecon{}_mixreal_FI220_v1.npy'.format(nfrecon))

In [ ]:
labsx,labsy = [], []
for i in range(ncombi):
    labsx.append(f'{oldcombi[i][0]}{oldcombi[i][1]}b0')
    for s in range(nbins):
        labsy.append(f'{oldcombi[i][0]}{oldcombi[i][1]}b{s}')


labs = []
for i in range(ncombi):
    labs.append(r'$\nu_{}\nu_{}$'.format(oldcombi[i][0], oldcombi[i][1]))
        
print(labs)

In [ ]:
ispectre = 2

plt.figure(figsize=(20, 20))
plt.yticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
plt.xticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)

for s in range(ncombi-1):
    plt.axhline(y=(s+1)*nbins-0.5, lw=3, color='k')
    plt.axvline(x=(s+1)*nbins-0.5, lw=3, color='k')
vmax = np.max(np.abs(covbinband[:, :, ispectre]))
# plt.imshow(covbinband[:, :, ispectre], vmin=-vmax, vmax=vmax, cmap='bwr')
vmax = np.max(np.abs(BBcov))
plt.imshow(BBcov, vmin=-vmax, vmax=vmax, cmap='bwr')
# plt.title('Covariance matrix (nfreq*nbins)x(nfreq*nbins) for {} spectrum'.format(clnames[ispectre]), 
#          fontsize=20)
plt.colorbar()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.figure(figsize=(15, 15))
ax = plt.gca()
plt.yticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
plt.xticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)


for s in range(ncombi-1):
    plt.axhline(y=(s+1)*nbins-0.5, lw=3, color='k')
    plt.axvline(x=(s+1)*nbins-0.5, lw=3, color='k')

# im = ax.imshow(corrbinband[:, :, ispectre], vmin=-1, vmax=1, cmap='bwr')
im = ax.imshow(BBcorr, vmin=-1, vmax=1, cmap='bwr')
plt.title('Correlation matrix (nfreq*nbins)x(nfreq*nbins) for {} spectrum'.format(clnames[ispectre]),
         fontsize=20)
plt.title(clnames[ispectre])

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# plt.savefig('/home/lmousset/QUBIC/Qubic_work/SpectroImagerie/paper_plot/IBCS_CorrMatrix_binfreq_FI150_GoodPhotonNoise_v4.pdf')

### Diference between same and mix

In [ ]:
nbands = 4
nbins = 16
BBcovmix = np.load(rep_save + 'BBcovariance_bincross_nfrecon{}_mixreal_FI-150.npy'.format(nbands))
BBcovsame = np.load(rep_save + 'BBcovariance_bincross_nfrecon{}_samereal_FI-150.npy'.format(nbands))

combi = list(combinations_with_replacement(np.arange(nbands), 2))
ncombi = len(combi)
labs = []
for i in range(ncombi):
    labs.append(f'{combi[i][0]}{combi[i][1]}')
    
plt.figure(figsize=(20, 10))
plt.subplot(131)
plt.yticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
plt.xticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
vmax = np.max(np.abs(2*BBcovmix))
plt.imshow(2*BBcovmix, vmin=-vmax, vmax=vmax, cmap='bwr')
plt.title('2 x Mix')
plt.colorbar(orientation='horizontal')


plt.subplot(132)
plt.yticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
plt.xticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
vmax = np.max(np.abs(BBcovsame))
plt.imshow(BBcovsame, vmin=-vmax, vmax=vmax, cmap='bwr')
plt.title('Same')
plt.colorbar(orientation='horizontal')

plt.subplot(133)
plt.yticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
plt.xticks(np.arange(8, nbins*ncombi, nbins), labs, fontsize=20)
vmax = np.max(np.abs(2 * BBcovmix - BBcovsame))
plt.imshow(2 * BBcovmix - BBcovsame, vmin=-vmax, vmax=vmax, cmap='bwr')
plt.title('2 x Mix - Same')
plt.colorbar(orientation='horizontal')


plt.tight_layout()

In [ ]:
# Plot the max of the covariance matrix vs nfrecon
# Not sure it makes sens
allrecon = [1, 2, 3, 4, 5, 8]
covmax = np.zeros((len(allrecon), 4))
for i, nfrecon in enumerate(allrecon[:-1]):
    print(i, nfrecon)
    covbinband = np.load(rep_save + f'BBcovariance_bincross_nfrecon{nfrecon}_mixreal.npy')
    print(covbinband.shape)
    print(np.max(covbinband))
    covmax[i, :] = np.max(np.diag(covbinband))
#     covmax[i, :] = covbinband[0, 0, :]

plt.figure(figsize=(12, 12))    
for s in range(4):
    plt.plot(allrecon[:], covmax[:, s]/covmax[0, s], 'o', label='{}'.format(clnames[s]))
plt.legend()
plt.axhline(y=1, color='k', ls='--')

## Weighted average of the subbands

The 3 methods (directly with Cp, with Cp_prime00 or with Cp_prime) give the same.

However, I don't get why the average band has this behaviour with bin...

In [ ]:
cls_cross.shape
Cp.shape
# cls_cross[:100, ...].shape

In [ ]:
# Using Cp directly
avg, sig2 = amc.make_weighted_av(cls_cross[:100, ...], Cp, verbose=False)
sig2MC = np.var(avg, axis=0)

# # Using Cp_prime 00
# avg00, sig200 = amc.make_weighted_av(cls_cross[:100, ...], Cp_prime00, verbose=False)
# sig2MC00 = np.var(avg00, axis=0)

# # Using Cp_prime
# avg_prime, sig2_prime = amc.make_weighted_av(cls_cross[:100, ...], Cp_prime, verbose=False)
# sig2MC_prime = np.var(avg_prime, axis=0)

In [ ]:
avg.shape
sig2.shape

In [ ]:
plt.figure(figsize=(10, 10))
s = 0
# for s in range(4):
plt.plot(sig2[:, s], color=colorlist[s], label='$\sigma^2$ {}'.format(clnames[s]))
# plt.plot(sig200[:, s], color=colorlist[s+1], label='$\sigma^2$ 00 {}'.format(clnames[s]))
# plt.plot(sig2_prime[:, s], color=colorlist[s+2], label='$\sigma^2$ prime {}'.format(clnames[s]))

plt.plot(sig2MC[:, s], 'o',color=colorlist[s], label='$\sigma^2$ MC {}'.format(clnames[s]))
# plt.plot(sig2MC00[:, s], 'o', color=colorlist[s+1], label='$\sigma^2$ MC 00 {}'.format(clnames[s]))
# plt.plot(sig2MC_prime[:, s], 'o', color=colorlist[s+2], label='$\sigma^2$ MC prime {}'.format(clnames[s]))
plt.legend()
plt.title('{} bands recombined in 1 band for {}'.format(nfrecon, clnames[s]))
plt.xlabel('bin')
plt.ylabel('$\sigma^2$')

In [ ]:
plt.figure(figsize=(10, 10))
s = 2
# for s in range(4):
plt.plot(ell_binned, np.sqrt(sig2[:, s]), color=colorlist[s], label='$\sigma$ {}'.format(clnames[s]))
# plt.plot(ell_binned, np.sqrt(sig200[:, s]), color=colorlist[s+1], label='$\sigma$ 00 {}'.format(clnames[s]))
# plt.plot(ell_binned, np.sqrt(sig2_prime[:, s]), color=colorlist[s+2], label='$\sigma$ prime {}'.format(clnames[s]))

plt.plot(ell_binned, np.sqrt(sig2MC[:, s]), 'o',color=colorlist[s], label='$\sigma$ MC {}'.format(clnames[s]))
# plt.plot(ell_binned, np.sqrt(sig2MC00[:, s]), 'o', color=colorlist[s+1], label='$\sigma$ MC 00 {}'.format(clnames[s]))
# plt.plot(ell_binned, np.sqrt(sig2MC_prime[:, s]), 'o', color=colorlist[s+2], label='$\sigma$ MC prime {}'.format(clnames[s]))
plt.legend()
plt.title('{} bands recombined in 1 band for {}'.format(nfrecon, clnames[s]))
plt.xlabel('$l$')
plt.ylabel('$\sigma$')

### Loop over nfrecon to get the recombined spectrum in each case

We will only do with Cp.

In [ ]:
allrecon = [1, 2, 3, 4, 5, 8]
allsig2 = np.zeros((len(allrecon), nbins, 4))
allsig2MC = np.zeros_like(allsig2)
avg = np.zeros((len(allrecon), 500, nbins, 4))
for i, nfrecon in enumerate(allrecon):
    print('\n nfrecon:', nfrecon)
#     cls_cross = np.load(rep_spectrumCC + 
#                     'cross_interband_mixreal_nfrecon{}_noisemaps_FI150_galaxycenter.npy'.format(nfrecon))

    cls_cross = np.load(rep_spectrumCC + 'spectrum_mixreal_FI-150_nfrecon{}.npy'.format(nfrecon))
    cls_cross = cls_cross[:500]
    print(cls_cross.shape)

    nreals = cls_cross.shape[0]
    nbins = cls_cross.shape[2]

    print('# realisations:', nreals)
    print('# bins:', nbins)
    
    Cp = amc.get_Cp(cls_cross, verbose=False)
#     Cp_check = np.zeros_like(Cp)
#     for s in range(4):
#         for b in range(nbins):
#             Cp_check[:, :, s, b] = Cp[:, :, s, b]
    
    avg[i, ...], allsig2[i, :, :] = amc.make_weighted_av(cls_cross, Cp, verbose=False)
    print(avg[i, ...].shape)
    allsig2MC[i, :, :] = np.var(avg[i, ...], axis=0)
  
    

In [ ]:
fig, axs = plt.subplots(2, 2)
axs = axs.ravel()

for i in range(4):
    ax = axs[i]
    for nbands in range(5, -1, -1):
        ax.errorbar(ell_binned[:], 
                np.mean(avg, axis=1)[nbands, :, i],
                yerr=np.std(avg, axis=1)[nbands, :, i] * np.sqrt(2),
                marker='o', linestyle='none', 
                label=f'Recombined spectrum {allrecon[nbands]} bands')

    ax.set_xlabel('$\\ell$')
    ax.set_ylabel('$D_\\ell$')
#     ax.set_yscale('log')
    ax.set_title(clnames[i])
    ax.grid()
    ax.legend(fontsize=8, loc='upper left')

plt.tight_layout()



In [ ]:
s = 2
plt.figure(figsize=(10, 10))
for i in range(6):
    plt.plot(ell_binned[:], allsig2[i, :, s] , label='nfrecon = {}'.format(allrecon[i]))

plt.xlabel('$l$')
plt.ylabel('$\sigma / \sigma_{1 band}$')
plt.title(clnames[s])
plt.legend()

In [ ]:
s = 2
thebin = 6
plt.figure(figsize=(10, 10))

plt.plot(allrecon, allsig2[:, thebin, s] / allsig2[0, thebin, s],
         label='bin = {}'.format(thebin))

plt.xlabel('# bands')
plt.ylabel('$\sigma$')
plt.legend()
# plt.axis('equal')
plt.grid()
print(allsig2[:, thebin, s])

In [ ]:
# Averaged over bins
plt.figure(figsize=(10, 10))
for s in range(4):
    plt.plot(allrecon, np.mean(allsig2, axis=1)[:, s]/np.mean(allsig2, axis=1)[0, s], label=clnames[s])
# plt.plot(allrecon, np.sqrt(allrecon), 'k', label='$\sqrt{N_{bands}}$')

plt.xlabel('# bands')
plt.ylabel('$\sigma / \sigma_{1band}$')
plt.legend()
plt.title('Avg over bins')

In [ ]:
mean = [0, 0]
mycov = [[1, -0.5], [-0.5, 1]]
plt.imshow(mycov)
x = np.random.multivariate_normal(mean, mycov, size=100000)
print(x.shape)

# Check we have the covariance we want
newcov = np.cov(x.T)
print(newcov)

# Look at the covariance of x²
x2 = x**2
newcov2 = np.cov(x2.T)
print(newcov2)